### Transform Customers data
- remove records with NULL customer_id
- remove excat duplicate records
- remove duplicate records based on created timestamp
- cast the columns to correct data type
- Write transformed data to silver schema

### remove records with NULL customer_id

In [0]:
select * from gizmobox.bronze.v_customers

In [0]:
select * from gizmobox.bronze.v_customers
where customer_id is not NULL
order by customer_id

### remove excat duplicate records


In [0]:
select distinct * from gizmobox.bronze.v_customers
where customer_id is not NULL
order by customer_id


### remove duplicate records based on created timestamp


In [0]:
with max_timestamp as (
  select customer_id, max(created_timestamp) as created_timestamp
  from gizmobox.bronze.v_customers
  group by customer_id
),
distinct_customers as (select distinct * from gizmobox.bronze.v_customers
where customer_id is not NULL
order by customer_id)
select d.*
from max_timestamp m
inner join distinct_customers d
on m.customer_id = d.customer_id
and m.created_timestamp = d.created_timestamp
order by customer_id



###  cast the columns to correct data type

In [0]:
with max_timestamp as (
  select customer_id, max(created_timestamp) as created_timestamp
  from gizmobox.bronze.v_customers
  group by customer_id
),
distinct_customers as (select distinct * from gizmobox.bronze.v_customers
where customer_id is not NULL
order by customer_id)
select CAST(d.created_timestamp AS TIMESTAMP),d.customer_id,d.customer_name, CAST(d.date_of_birth as DATE),d.email,
CAST(d.member_since as DATE),d.telephone, d.file_path
from max_timestamp m
inner join distinct_customers d
on m.customer_id = d.customer_id
and m.created_timestamp = d.created_timestamp
order by customer_id

###  Write transformed data to silver schema

In [0]:
CREATE TABLE IF NOT EXISTS gizmobox.silver.customers AS
with max_timestamp as (
  select customer_id, max(created_timestamp) as created_timestamp
  from gizmobox.bronze.v_customers
  group by customer_id
),
distinct_customers as (select distinct * from gizmobox.bronze.v_customers
where customer_id is not NULL
order by customer_id)
select CAST(d.created_timestamp AS TIMESTAMP),d.customer_id,d.customer_name, CAST(d.date_of_birth as DATE),d.email,
CAST(d.member_since as DATE),d.telephone, d.file_path
from max_timestamp m
inner join distinct_customers d
on m.customer_id = d.customer_id
and m.created_timestamp = d.created_timestamp
order by customer_id

In [0]:
select * from gizmobox.silver.customers

In [0]:
DESCRIBE EXTENDED gizmobox.silver.customers